## Exploration of LRP with binary MNIST

**Function        : Exploration of LRP with binary MNIST**<br>
**Author          : Team DIANNA**<br>
**Contributor     :**<br>
**First Built     : 2021.07.13**<br>
**Last Update     : 2021.09.03**<br>
**Library         : os, numpy, matplotlib, torch, captum**<br>
**Description     : In this notebook we test XAI method LRP using trained binary MNIST model.**<br>
**Return Values   : Relevance scores**<br>
**Note**          : We use Captum library to perform LRP. This library works only with pytorch and it is not compitable with onnx.<br>

In [ ]:
%matplotlib inline
import pathlib
import time as tt
import numpy as np
# DL framework
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
#import torch.onnx
# XAI framework
from captum.attr import LRP
from captum.attr import visualization as viz
# for plotting
import matplotlib.pyplot as plt

### Path to the dataset and the model

In [ ]:
# please specify data path
datapath = pathlib.Path.home() / 'SURFdrive/Shared/datasets/mnist'
# please specify model path
model_path = pathlib.Path.cwd() / '../../model_generation/MNIST'

### Load data (binary MNIST)

In [ ]:
# load binary MNIST from local
# load data
fd = np.load(datapath / 'binary-mnist.npz')
# training set
train_X = fd['X_train']
train_y = fd['y_train']
# testing set
test_X = fd['X_test']
test_y = fd['y_test']
fd.close()

# dimensions of data
print("dimensions of mnist:")
print("dimensions or training set", train_X.shape)
print("dimensions or training set label", train_y.shape)
print("dimensions or testing set", test_X.shape)
print("dimensions or testing set label", test_y.shape)
# statistics of training set
print("statistics of training set:")
print("Digits: 0 1")
print("labels: {}".format(np.unique(train_y)))
print("Class distribution: {}".format(np.bincount(train_y)))
print("First few labels of training set", train_y[:20])

### Prepare data as torch tensor

In [ ]:
# use pytorch data loader
test_X_torch = torch.from_numpy(test_X).type(torch.FloatTensor)
test_y_torch = torch.from_numpy(test_y).type(torch.LongTensor)
# reshape the input following the definition in pytorch (batch, channel, Height, Width)
test_X_torch = test_X_torch.view(-1,1,28,28)

### Load model (Pytorch model trained for binary MNIST)

In [ ]:
# define the model first
class MnistNet(nn.Module):
    def __init__(self, kernels=[16, 32], dropout = 0.1, classes=2):
        '''
        Two layer CNN model with max pooling.
        '''
        super(MnistNet, self).__init__()
        self.kernels = kernels
        # 1st layer
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Dropout()
        )
        # 2nd layer
        self.layer2 = nn.Sequential(
            nn.Conv2d(kernels[0], kernels[1], kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Dropout()
        )
        self.fc1 = nn.Linear(7 * 7 * kernels[-1], kernels[-1]) # pixel 28 / maxpooling 2 * 2 = 7
        self.fc2 = nn.Linear(kernels[-1], classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

# hyper-parameters
kernels = [16, 32]
dropout = 0.5
classes = 2
# create model
model = MnistNet(kernels, dropout, classes)
# load whole model state
checkpoint = torch.load(model_path / 'mnistnet_training_checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])

### Predict the class of the input image <br>

In [ ]:
# check the prediction
model.eval()

# overall test accuracy
correct = 0
for i in range(len(test_X_torch)):
    output = model(test_X_torch[i:i+1,:,:,:])
    predicted = torch.max(output,1)[1]
    correct += (predicted == test_y[i]).sum()

print("Test accuracy:{:.3f}% ".format(float(correct*100) / float(len(test_X_torch))))

# check one case
output = model(test_X_torch[:1,:,:,:])
predicted = torch.max(output,1)[1]
print("prediction", predicted)
print("ground truth", test_y[0])

### LRP-based attribution <br>
Compute attributions using LRP and visualize them on the image. <br>
Layer-wise relevance propagation is based on a backward propagation mechanism applied sequentially to all layers of the model. The model output score represents the initial relevance which is decomposed into values for each neuron of the underlying layers.<br>

In [ ]:
lrp = LRP(model)

case = 10
for i in range(case):
    # note that the model is so well trained and the output can sometimes be 0 
    # (which means the prediction is close to 1, given the output calculated as log_softmax)
    # This will cause an error for this implementation. To avoid this, we check the other class (`target=(1-test_y_torch[i])`),
    # which is equivalent since the results all indicate the relevance pixel for the prediction (two sides of one coin)
    attributions_lrp = lrp.attribute(test_X_torch[i:i+1,:,:,:], target=(1-test_y_torch[i]))
    # display attraibution map alongside with the predictand
    _ = viz.visualize_image_attr_multiple(np.transpose(attributions_lrp[0,:,:,:].cpu().detach().numpy(), (1,2,0)),
                                          np.transpose(test_X_torch[i,:,:,:].cpu().detach().numpy(), (1,2,0)),
                                          ["original_image", "blended_heat_map"],
                                          ["all", "absolute_value"],
                                          show_colorbar=True)

### Conclusion
The above examples are pixel-wise decompositions for a multilayer neural network trained and tested on MNIST digits, using layer-wise relevance propagation. These heat maps show the pixels that support the classification. Pixels/super pixels with higher relevance scores indicate higher importance as the evidence for being the predicted class.<br>

The results suggest that pixels "inside" the digits are important for the predictions.

In [ ]:
i = 0
case0 = np.transpose(test_X_torch[i,:,:,:].cpu().detach().numpy(), (1,2,0))
attributions_lrp = lrp.attribute(test_X_torch[i:i+1,:,:,:], target=(1-test_y_torch[i]))
attributions0 = np.transpose(attributions_lrp[0,:,:,:].cpu().detach().numpy(), (1,2,0))

In [ ]:
attributions0.shape

In [ ]:
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(10,6))
character_plot = ax[0].imshow(case0[...,0])
fig.colorbar(character_plot, ax=ax[0], shrink=0.67)

# determine value limits to center colorbar on zero
vmax = max(attributions0[...,0].max(), abs(attributions0[...,0].min()))
vmin = -vmax

attributions_plot = ax[1].imshow(attributions0[...,0], cmap='bwr_r', vmin=vmin, vmax=vmax)
fig.colorbar(attributions_plot, ax=ax[1], shrink=0.67)

_ = viz.visualize_image_attr_multiple(np.transpose(attributions_lrp[0,:,:,:].cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(test_X_torch[i,:,:,:].cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "blended_heat_map"],
                                      ["all", "absolute_value"],
                                      show_colorbar=True)

### Conclusion 2
I wanted to check out the raw attributions numbers, visualize them myself to see whether they made more sense to me than with the Captum visualization. Unfortunately, it does not. On the one hand, the 0's make more sense than the 1's in the Captum viz, but this could just be a manufactured visualization choice in Captum as well. The non-zero pixels don't make much sense. Why are some red and some blue? No idea.